In [4]:
from google.colab import drive
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [5]:
import tarfile
import os
import nibabel as nib
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import tensorflow 
import seaborn as sns
import pickle
import random

In [6]:
train_data_path = "/content/gdrive/MyDrive/BraTSProject/data/BraTS2021_Training_Data"

In [7]:
if(nib.load(train_data_path+"/BraTS2021_00000/BraTS2021_00000_flair.nii.gz")):
  image_size=nib.load(train_data_path+"/BraTS2021_00000/BraTS2021_00000_flair.nii.gz").get_fdata().shape[0]
  print("Data path working...")
  print(image_size)
else:
  print("Data path failed. Make sure your data_path is correct")

Data path working...
240


In [28]:

DATA_PATH = "/content/gdrive/MyDrive/BraTSProject/data/BraTS2021_Training_Data"
TRANSLATED_4PATH = "/content/gdrive/MyDrive/BraTSProject/Classifier/4classification.pickle"
FRONT_LEFT  = [1, 0, 0, 0]
FRONT_RIGHT = [0, 1, 0, 0]
BACK_LEFT   = [0, 0, 1, 0]
BACK_RIGHT  = [0, 0, 0, 1]


def load_image(path, downsampling_factor=1):
    volume = nib.load(path).get_fdata()
    if downsampling_factor != 1:
        new_shape = np.array(volume.shape) // downsampling_factor
        volume = zoom(volume, new_shape / np.array(volume.shape), order=3)
    return volume

def load_image_paths():
    image_paths = []
    for root, dirs, files in os.walk(DATA_PATH):
        for file in files:
            if('flair.nii' in file):
                image_paths.append(os.path.join(root, file))
    return image_paths

def load_segmentation_paths():
    image_paths = []
    for root, dirs, files in os.walk(DATA_PATH):
        for file in files:
            if('seg.nii' in file):
                image_paths.append(os.path.join(root, file))
    return image_paths

def load_images():
    images = []
    for root, dirs, files in os.walk(DATA_PATH):
        for file in files:
            if('flair.nii' in file):
                images.append(load_image(os.path.join(root, file)))
    return images

def load_segmentations():
    segmentations = []
    for root, dirs, files in os.walk(DATA_PATH):
        for file in files:
            if('seg.nii' in file):
                segmentations.append(load_image(os.path.join(root, file)))
    return segmentations

def load_classifications():
    return pickle.load(TRANSLATED_4PATH)

# Translate a label and image from a segmentation to the region where the tumor is.
def translate_label(image, label):
    tumor_indices = np.argwhere(label > 0)
    brain_indices = np.argwhere(image > 0)
    tumor_center_of_mass = tumor_indices.mean(axis=0)
    brain_center_of_mass = brain_indices.mean(axis=0)
    if(tumor_center_of_mass[1] < brain_center_of_mass[1]):
        if(tumor_center_of_mass[2] < brain_center_of_mass[2]):
            return FRONT_LEFT
        else:
            return BACK_LEFT
    else:
        if(tumor_center_of_mass[2] < brain_center_of_mass[2]):
            return FRONT_RIGHT
        else:
            return BACK_RIGHT

def save_4classification():
    img_paths = load_image_paths()
    label_paths = load_segmentation_paths()
    N_IMGS = len(img_paths)
    translated = []
    for i in range(N_IMGS):
        img = load_image(img_paths[i])
        lbl = load_image(label_paths[i])
        t_lbl = translate_label(img, lbl)
        translated.append(t_lbl)
    with open(TRANSLATED_4PATH, 'wb') as f:
        pickle.dump(translated, f)

def load_4classification():
    labels = None
    with open(TRANSLATED_4PATH, 'rb') as f:
        labels = pickle.load(f)
    return labels


In [29]:
classification_labels = load_4classification()

print(f"FRONT LEFT {classification_labels.count(FRONT_LEFT)}")
print(f"FRONT RIGHT {classification_labels.count(FRONT_RIGHT)}")
print(f"FRONT LEFT {classification_labels.count(BACK_LEFT)}")
print(f"BACK RIGHT {classification_labels.count(BACK_RIGHT)}")

FRONT LEFT 301
FRONT RIGHT 215
FRONT LEFT 378
BACK RIGHT 357


In [30]:
from scipy.ndimage import rotate
from tensorflow.keras.utils import Sequence
class DataGenerator(Sequence):
    def __init__(self, image_paths, labels, batch_size):
        self.image_paths = image_paths
        self.labels = labels
        self.batch_size = batch_size // 2 # adjusting for later insertion of augmented images
        self.image_shape = load_image(self.image_paths[0]).shape

    def __len__(self):
        return int(np.ceil(len(self.image_paths) / float(self.batch_size)))

    def __getitem__(self, idx):
        batch_image_paths = self.image_paths[idx * self.batch_size: (idx + 1) * self.batch_size]
        blabels = self.labels[idx * self.batch_size: (idx + 1) * self.batch_size]

        batch_images = np.zeros((2 * self.batch_size, *self.image_shape))  # Double the size to include original and augmented images
        batch_labels = np.zeros((2 * self.batch_size, 4))

        for i, image_path in enumerate(batch_image_paths):
            original_image = load_image(image_path, downsampling_factor=2)
            augmented_image = self.augment_image(np.copy(original_image))

            batch_images[2 * i] = original_image
            batch_images[2 * i + 1] = augmented_image
            batch_labels[2 * i] = blabels[i]
            batch_labels[2 * i + 1] = blabels[i]

        return batch_images, batch_labels

    def augment_image(self, image):
        # Flip
        if random.random() < 0.5:
            image = np.flip(image, axis=random.choice([0, 1]))

        # Rotate
        angle = random.uniform(-20, 20)
        axes = random.choice([(0, 1), (1, 2), (0, 2)])
        image = rotate(image, angle, axes=axes, reshape=False, mode='nearest')

        # Adjust brightness
        brightness_factor = random.uniform(0.8, 1.2)
        image = np.clip(image * brightness_factor, 0, 255)

        return image[:, :, :, 64]

In [31]:
from tensorflow.keras import layers, models
def create_model():
    model = models.Sequential()
    model.add(layers.Conv3D(16, (3, 3, 3), activation='relu', input_shape=(240, 240, 64, 1)))
    model.add(layers.MaxPooling3D((2, 2, 2)))
    model.add(layers.Conv3D(32, (3, 3, 3), activation='relu'))
    model.add(layers.MaxPooling3D((2, 2, 2)))
    model.add(layers.Conv3D(32, (3, 3, 3), activation='relu'))

    model.add(layers.Flatten())
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(4, activation='softmax'))

    return model


def main():
    
    image_paths = load_image_paths()
    labels = np.array(load_4classification())

    N_IMAGES = len(image_paths)
    N_TRAIN = (N_IMAGES // 4)

    train_image_paths, test_image_paths = image_paths[:N_TRAIN], image_paths[N_TRAIN:]
    train_labels, test_labels = labels[:N_TRAIN], labels[N_TRAIN:]


    
    # Test using less then the whole dataset
    '''
    train_image_paths = train_image_paths[:(len(train_image_paths) // 4)]
    test_image_paths = test_image_paths[:(len(test_image_paths) // 4)]
    train_labels = train_labels[:(len(train_labels) // 4)]
    test_labels = test_labels[:(len(test_labels) // 4)]
    '''

    train_generator = DataGenerator(train_image_paths, train_labels, 3)
    test_generator = DataGenerator(test_image_paths, test_labels, 3)

    model = create_model()
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    model.fit(train_generator, epochs=5, validation_data=test_generator)
    model.save('/content/gdrive/MyDrive/BraTSProject/Classifier')
    test_loss, test_acc = model.evaluate(test_generator, verbose=2)
    print('Test accuracy:', test_acc)



In [32]:
main()

NameError: ignored